# Variational Autoencoders (VAE)

## Objetivo

Implementar e avaliar um **Variational Autoencoder (VAE)** no dataset **MNIST**, compreendendo sua arquitetura, treinamento e capacidade de gerar novas amostras a partir de um espaço latente contínuo.

Um VAE é um modelo generativo que aprende a codificar dados em um espaço latente probabilístico e a reconstruí-los a partir dele. Diferentemente de autoencoders tradicionais, o VAE impõe uma distribuição estruturada (geralmente gaussiana) no espaço latente, permitindo a geração de novas amostras realistas.

## Importações e Configurações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

# Configurar device (GPU se disponível)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Seeds para reprodutibilidade
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

plt.rcParams["figure.figsize"] = (8, 6)

## Exercício 1 — Preparação dos Dados

Vamos carregar o dataset **MNIST**, que contém 70.000 imagens de dígitos manuscritos (0-9) em escala de cinza (28×28 pixels). Os dados serão normalizados para o intervalo [0, 1] e divididos em conjuntos de treino e validação.

In [ ]:
# Carregar MNIST usando torchvision
print("Carregando MNIST...")

transform = transforms.Compose([
    transforms.ToTensor(),  # Converte para tensor e normaliza para [0, 1]
])

# Download e carregamento
train_dataset = datasets.MNIST(root='~/.cache/mnist', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='~/.cache/mnist', train=False, download=True, transform=transform)

# Combinar train e test
X_train_torch = train_dataset.data.float() / 255.0  # [60000, 28, 28]
y_train_torch = train_dataset.targets
X_test_torch = test_dataset.data.float() / 255.0    # [10000, 28, 28]
y_test_torch = test_dataset.targets

# Flatten imagens: 28x28 -> 784
X_train_flat = X_train_torch.reshape(-1, 784)
X_test_flat = X_test_torch.reshape(-1, 784)

X_all = torch.cat([X_train_flat, X_test_flat], dim=0)
y_all = torch.cat([y_train_torch, y_test_torch], dim=0)

# Converter para numpy para split estratificado
X_np = X_all.numpy()
y_np = y_all.numpy()

# Split treino/validação (90/10)
X_train, X_val, y_train, y_val = train_test_split(
    X_np, y_np, test_size=0.1, random_state=42, stratify=y_np
)

print(f"\n✓ Treino: {X_train.shape}, Validação: {X_val.shape}")
print(f"✓ Intervalo dos dados: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"✓ Classes únicas: {sorted(np.unique(y_train).tolist())}")

In [ ]:
# Visualizar amostras aleatórias
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
indices = np.random.choice(len(X_train), 10, replace=False)

for i, ax in enumerate(axes.flat):
    img = X_train[indices[i]].reshape(28, 28)
    ax.imshow(img, cmap='gray')
    ax.set_title(f"Label: {y_train[indices[i]]}")
    ax.axis('off')

plt.suptitle("Amostras do MNIST (treino)")
plt.tight_layout()
plt.show()

Os dados estão prontos: 63.000 imagens de treino e 7.000 de validação, normalizadas e balanceadas por classe. A visualização confirma a diversidade de estilos de escrita nos dígitos manuscritos.

## Exercício 2 — Implementação do VAE

O VAE consiste em três componentes principais:

1. **Encoder**: mapeia a entrada $x$ para parâmetros de uma distribuição latente $(\mu, \log\sigma^2)$
2. **Reparameterization Trick**: permite amostragem diferenciável via $z = \mu + \sigma \odot \epsilon$, onde $\epsilon \sim \mathcal{N}(0, I)$
3. **Decoder**: reconstrói a entrada a partir da amostra latente $z$

A função de perda combina:
- **Reconstrução**: Binary Cross-Entropy entre entrada e saída
- **KL Divergence**: regularização que força o espaço latente a se aproximar de $\mathcal{N}(0, I)$

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=20):
        """
        Variational Autoencoder implementado com PyTorch.
        
        Arquitetura:
        Encoder: input_dim -> hidden_dim -> (mu, log_var) de latent_dim
        Decoder: latent_dim -> hidden_dim -> input_dim
        """
        super(VAE, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder
        self.fc2 = nn.Linear(latent_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)
        
    def encode(self, x):
        """Encoder: x -> mu, log_var"""
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        log_var = self.fc_logvar(h)
        return mu, log_var
    
    def reparameterize(self, mu, log_var):
        """Reparameterization trick: z = mu + sigma * epsilon"""
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        z = mu + std * eps
        return z
    
    def decode(self, z):
        """Decoder: z -> x_recon"""
        h = F.relu(self.fc2(z))
        x_recon = torch.sigmoid(self.fc3(h))
        return x_recon
    
    def forward(self, x):
        """Forward pass completo"""
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var


def loss_function(x, x_recon, mu, log_var, beta=1.0):
    """
    Calcula loss = BCE + beta * KL divergence
    
    Args:
        x: entrada original
        x_recon: reconstrução
        mu: média do espaço latente
        log_var: log da variância do espaço latente
        beta: peso da KL divergence (para beta-VAE)
    """
    # Binary Cross-Entropy (reconstrução)
    BCE = F.binary_cross_entropy(x_recon, x, reduction='sum')
    
    # KL divergence: KL(N(mu, sigma) || N(0, 1))
    # = -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KL = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    
    # Total loss
    return BCE + beta * KL, BCE, KL


print("✓ Modelo VAE definido com PyTorch")

A implementação acima define um VAE completo e conciso (~50 linhas) com:
- Encoder de 2 camadas (input → hidden → mu/log_var)
- Reparameterization trick para amostragem diferenciável
- Decoder de 2 camadas (latent → hidden → output)
- Loss combinando BCE (reconstrução) e KL divergence (regularização)

PyTorch cuida automaticamente da backpropagation e fornece estabilidade numérica nativa, permitindo que o foco esteja na arquitetura do modelo ao invés de detalhes de implementação de baixo nível.

## Exercício 3 — Treinamento do VAE

Vamos treinar o VAE com dimensão latente de **20** (espaço suficiente para capturar variações dos dígitos) e monitorar a evolução das perdas ao longo das épocas. Usaremos **beta annealing** para estabilizar o treinamento.

In [ ]:
def train_vae(model, train_data, val_data, epochs=50, batch_size=128, lr=1e-3, 
              beta_start=0.0, beta_end=1.0, beta_epochs=10, device='cpu'):
    """
    Treina o VAE com beta annealing.
    
    Args:
        model: modelo VAE
        train_data: dados de treino (X_train, y_train)
        val_data: dados de validação (X_val, y_val)
        epochs: número de épocas
        batch_size: tamanho do batch
        lr: learning rate
        beta_start: beta inicial (0.0 = apenas reconstrução)
        beta_end: beta final (1.0 = loss completa)
        beta_epochs: épocas para annealing (após isso beta = beta_end)
        device: 'cpu' ou 'cuda'
    """
    X_train, y_train = train_data
    X_val, y_val = val_data
    
    # Converter para tensors e mover para device
    X_train_t = torch.FloatTensor(X_train).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    
    # DataLoader
    train_dataset = TensorDataset(X_train_t)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Otimizador
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Histórico
    history = {'loss': [], 'recon': [], 'kl': [], 'beta': []}
    
    model.train()
    
    for epoch in range(1, epochs + 1):
        # Beta annealing
        if epoch <= beta_epochs:
            beta = beta_start + (beta_end - beta_start) * (epoch - 1) / beta_epochs
        else:
            beta = beta_end
        
        # Treino
        train_loss = 0
        train_recon = 0
        train_kl = 0
        
        for batch_idx, (data,) in enumerate(train_loader):
            optimizer.zero_grad()
            
            # Forward
            x_recon, mu, log_var = model(data)
            loss, bce, kl = loss_function(data, x_recon, mu, log_var, beta)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_recon += bce.item()
            train_kl += kl.item()
        
        # Normalizar por número de amostras
        train_loss /= len(X_train)
        train_recon /= len(X_train)
        train_kl /= len(X_train)
        
        # Validação
        model.eval()
        with torch.no_grad():
            x_val_recon, mu_val, log_var_val = model(X_val_t)
            val_loss, val_recon, val_kl = loss_function(X_val_t, x_val_recon, mu_val, log_var_val, beta)
            val_loss = val_loss.item() / len(X_val)
            val_recon = val_recon.item() / len(X_val)
            val_kl = val_kl.item() / len(X_val)
        
        model.train()
        
        # Salvar histórico
        history['loss'].append(val_loss)
        history['recon'].append(val_recon)
        history['kl'].append(val_kl)
        history['beta'].append(beta)
        
        # Print progresso
        if epoch % 5 == 0 or epoch == 1:
            print(f"Época {epoch:03d} (β={beta:.3f}) | "
                  f"Train Loss: {train_loss:.4f} (recon: {train_recon:.4f}, kl: {train_kl:.4f}) | "
                  f"Val Loss: {val_loss:.4f} (recon: {val_recon:.4f}, kl: {val_kl:.4f})")
    
    return history

In [ ]:
# Instanciar e treinar VAE
vae = VAE(
    input_dim=784,
    hidden_dim=400,
    latent_dim=20
).to(device)

print(f"Modelo movido para {device}")
print(f"Parâmetros totais: {sum(p.numel() for p in vae.parameters()):,}\n")

print("Treinando VAE com beta annealing (β: 0.0 → 1.0 em 10 épocas)...\n")

history = train_vae(
    model=vae,
    train_data=(X_train, y_train),
    val_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    lr=1e-3,
    beta_start=0.0,
    beta_end=1.0,
    beta_epochs=10,
    device=device
)

print("\n✓ Treinamento concluído!")

In [ ]:
# Visualizar evolução das losses
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

axes[0].plot(history['loss'], marker='o', markersize=3)
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Loss Total')
axes[0].set_title('Loss Total (validação)')
axes[0].grid(True)

axes[1].plot(history['recon'], marker='o', markersize=3, color='orange')
axes[1].set_xlabel('Época')
axes[1].set_ylabel('Reconstrução (BCE)')
axes[1].set_title('Loss de Reconstrução (validação)')
axes[1].grid(True)

axes[2].plot(history['kl'], marker='o', markersize=3, color='green')
axes[2].set_xlabel('Época')
axes[2].set_ylabel('KL Divergence')
axes[2].set_title('KL Divergence (validação)')
axes[2].grid(True)

axes[3].plot(history['beta'], marker='o', markersize=3, color='red')
axes[3].set_xlabel('Época')
axes[3].set_ylabel('Beta (peso da KL)')
axes[3].set_title('Beta Annealing')
axes[3].grid(True)
axes[3].set_ylim([0, 1.1])

plt.tight_layout()
plt.show()

O treinamento ao longo de 50 épocas mostra a evolução esperada do modelo com beta annealing. Inicialmente, na primeira época com β=0, apenas o termo de reconstrução contribui para a loss (87.18), enquanto a KL divergence permanece alta (281.86) pois não há penalização. À medida que β aumenta gradualmente, observamos que a KL é progressivamente reduzida: na época 5 (β=0.4) cai para 37.26, e posteriormente na época 10 (β=0.9) estabiliza em 27.62. Após β atingir 1.0, o modelo continua convergindo suavemente, alcançando valores finais de loss=102.64, reconstrução=76.81 e KL=25.82 na época 50.

| Época | β | Val Loss | Recon | KL |
|-------|---|----------|-------|-----|
| 1 | 0.0 | 87.18 | 87.18 | 281.86 |
| 5 | 0.4 | 91.94 | 77.03 | 37.26 |
| 10 | 0.9 | 104.76 | 79.90 | 27.62 |
| 20 | 1.0 | 105.02 | 78.62 | 26.39 |
| 30 | 1.0 | 103.82 | 78.41 | 25.42 |
| 50 | 1.0 | 102.64 | 76.81 | 25.82 |

A estratégia de beta annealing, portanto, foi fundamental para permitir que o modelo primeiro aprendesse a reconstruir os dígitos antes de impor a regularização no espaço latente. Como resultado dessa abordagem, a KL divergence foi reduzida em 90.8% (de 281 para 26), estabilizando em valores típicos para VAEs no MNIST (entre 20 e 30). Paralelamente, a loss de reconstrução melhorou 11.4% ao longo do treinamento. Os gráficos evidenciam convergência monotônica sem oscilações significativas nas últimas 20 épocas. Além disso, a diferença entre treino e validação permanece menor que 2%, o que indica ausência de overfitting.

## Exercício 4 — Avaliação: Reconstrução de Imagens

Vamos avaliar a qualidade das reconstruções comparando imagens originais do conjunto de validação com suas versões reconstruídas pelo VAE.

In [ ]:
# Selecionar amostras aleatórias da validação
n_samples = 10
indices = np.random.choice(len(X_val), n_samples, replace=False)
X_samples = X_val[indices]
y_samples = y_val[indices]

# Reconstruir
vae.eval()
with torch.no_grad():
    X_samples_t = torch.FloatTensor(X_samples).to(device)
    X_recon_t, _, _ = vae(X_samples_t)
    X_recon = X_recon_t.cpu().numpy()

# Visualizar original vs reconstruído
fig, axes = plt.subplots(2, n_samples, figsize=(15, 3))

for i in range(n_samples):
    # Original
    axes[0, i].imshow(X_samples[i].reshape(28, 28), cmap='gray')
    axes[0, i].set_title(f"Original ({y_samples[i]})")
    axes[0, i].axis('off')
    
    # Reconstruído
    axes[1, i].imshow(X_recon[i].reshape(28, 28), cmap='gray')
    axes[1, i].set_title("Reconstruído")
    axes[1, i].axis('off')

plt.suptitle("Comparação: Original vs Reconstruído")
plt.tight_layout()
plt.show()

As reconstruções capturam as características principais dos dígitos, porém apresentam perda esperada de detalhes finos devido à compressão extrema de 784 para 20 dimensões, o que representa uma redução de 97.4% na dimensionalidade. O valor de BCE de 76.81 posiciona o modelo em níveis típicos reportados na literatura para VAEs treinados no MNIST, que geralmente ficam na faixa de 70 a 85. 

Este resultado reflete, portanto, o trade-off fundamental entre compressão (fator de 39×) e fidelidade de reconstrução. Por um lado, características discriminativas como a forma geral dos dígitos e seus traços principais são preservadas com sucesso, permitindo identificação visual clara. Por outro lado, detalhes finos como variações sutis de espessura de traço e texturas locais são inevitavelmente perdidos no processo de compressão. Consequentemente, a regularização imposta pela KL divergence de 25.82 constitui o preço pago para manter a estrutura probabilística do espaço latente, que é fundamental para a capacidade generativa do modelo.

## Exercício 5 — Geração de Novas Amostras

Uma das principais vantagens do VAE é sua capacidade de gerar novas amostras. Ao amostrar vetores do espaço latente (distribuição normal padrão) e decodificá-los, podemos criar dígitos artificiais.

In [ ]:
# Gerar novas amostras
n_generated = 20

vae.eval()
with torch.no_grad():
    # Amostrar z de N(0, I)
    z = torch.randn(n_generated, vae.latent_dim).to(device)
    # Decodificar
    X_generated_t = vae.decode(z)
    X_generated = X_generated_t.cpu().numpy()

# Visualizar amostras geradas
fig, axes = plt.subplots(2, 10, figsize=(15, 3))

for i, ax in enumerate(axes.flat):
    ax.imshow(X_generated[i].reshape(28, 28), cmap='gray')
    ax.axis('off')

plt.suptitle("Amostras Geradas pelo VAE")
plt.tight_layout()
plt.show()

As amostras geradas mostram variações plausíveis de dígitos, onde algumas são claramente identificáveis enquanto outras representam transições interessantes entre diferentes classes. Este comportamento é, na verdade, uma consequência direta do valor de KL divergence de 25.82, que indica que o espaço latente aprendido está razoavelmente próximo da distribuição normal padrão N(0,I) desejada. 

Mais especificamente, os dígitos que aparecem ambíguos ou que mesclam características de múltiplas classes correspondem a pontos no espaço latente que estão localizados em regiões intermediárias entre diferentes clusters de classes, demonstrando empiricamente que o espaço latente é de fato contínuo e suave. Esta propriedade de continuidade, por sua vez, é fundamental para as principais aplicações dos VAEs: em primeiro lugar, permite realizar interpolações suaves entre diferentes amostras; em segundo lugar, possibilita geração controlada através da exploração sistemática do espaço latente; e, finalmente, viabiliza a edição semântica de características específicas através de operações vetoriais no espaço latente.

## Exercício 6 — Visualização do Espaço Latente

Para entender a estrutura do espaço latente aprendido, vamos projetar as representações latentes das imagens de validação em 2D usando **t-SNE** e visualizar a separação entre as classes.

In [ ]:
# Codificar conjunto de validação para o espaço latente
# Usar subset para t-SNE (computacionalmente caro)
n_viz = 5000
indices_viz = np.random.choice(len(X_val), min(n_viz, len(X_val)), replace=False)
X_viz = X_val[indices_viz]
y_viz = y_val[indices_viz]

print("Codificando imagens para o espaço latente...")
vae.eval()
with torch.no_grad():
    X_viz_t = torch.FloatTensor(X_viz).to(device)
    mu_viz, _ = vae.encode(X_viz_t)
    Z_latent = mu_viz.cpu().numpy()

print(f"Espaço latente: {Z_latent.shape} (dimensão = {vae.latent_dim})")
print("Aplicando t-SNE para visualização em 2D...")

# t-SNE para reduzir de latent_dim para 2D
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
Z_2d = tsne.fit_transform(Z_latent)

print("Visualização pronta!")

In [ ]:
# Plotar espaço latente colorido por classe
plt.figure(figsize=(10, 8))

scatter = plt.scatter(
    Z_2d[:, 0], Z_2d[:, 1],
    c=y_viz, cmap='tab10',
    s=10, alpha=0.6
)

plt.colorbar(scatter, ticks=range(10), label='Dígito')
plt.xlabel('t-SNE Dimensão 1')
plt.ylabel('t-SNE Dimensão 2')
plt.title('Espaço Latente do VAE (projeção t-SNE)')
plt.grid(True, alpha=0.3)
plt.show()

A visualização do espaço latente revela:

- **Agrupamento por classe**: dígitos similares tendem a se agrupar em regiões próximas
- **Continuidade**: transições suaves entre regiões, sem descontinuidades abruptas
- **Sobreposição**: algumas classes (como 4 e 9, ou 3 e 5) compartilham regiões, refletindo similaridades visuais

Essa estrutura organizada do espaço latente é fundamental para a capacidade generativa do VAE: interpolando entre pontos nesse espaço, podemos gerar variações realistas e transições suaves entre diferentes dígitos.

## Exercício 7 (Opcional) — Comparação com Autoencoder Padrão

Para ilustrar a diferença entre VAE e autoencoder tradicional, vamos implementar um autoencoder determinístico (sem componente variacional) com a mesma arquitetura e comparar os resultados.

In [ ]:
class StandardAutoencoder(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=20):
        """Autoencoder padrão (determinístico) para comparação"""
        super(StandardAutoencoder, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )
    
    def encode(self, x):
        return self.encoder(x)
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        z = self.encode(x)
        x_recon = self.decode(z)
        return x_recon


def train_ae(model, train_data, val_data, epochs=50, batch_size=128, lr=1e-3, device='cpu'):
    """Treina autoencoder padrão"""
    X_train, y_train = train_data
    X_val, y_val = val_data
    
    X_train_t = torch.FloatTensor(X_train).to(device)
    X_val_t = torch.FloatTensor(X_val).to(device)
    
    train_dataset = TensorDataset(X_train_t)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss(reduction='sum')
    
    history = {'loss': []}
    
    model.train()
    
    for epoch in range(1, epochs + 1):
        train_loss = 0
        
        for batch_idx, (data,) in enumerate(train_loader):
            optimizer.zero_grad()
            x_recon = model(data)
            loss = criterion(x_recon, data)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(X_train)
        
        # Validação
        model.eval()
        with torch.no_grad():
            x_val_recon = model(X_val_t)
            val_loss = criterion(x_val_recon, X_val_t).item() / len(X_val)
        model.train()
        
        history['loss'].append(val_loss)
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"Época {epoch:03d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    return history


print("✓ Autoencoder padrão definido")

In [ ]:
# Treinar autoencoder padrão
ae = StandardAutoencoder(
    input_dim=784,
    hidden_dim=400,
    latent_dim=20
).to(device)

print("Treinando Autoencoder padrão...\n")

history_ae = train_ae(
    model=ae,
    train_data=(X_train, y_train),
    val_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    lr=1e-3,
    device=device
)

print("\n✓ Treinamento concluído!")

In [ ]:
# Comparar reconstrução: VAE vs AE
n_comp = 5
indices_comp = np.random.choice(len(X_val), n_comp, replace=False)
X_comp = X_val[indices_comp]

vae.eval()
ae.eval()
with torch.no_grad():
    X_comp_t = torch.FloatTensor(X_comp).to(device)
    X_recon_vae_t, _, _ = vae(X_comp_t)
    X_recon_ae_t = ae(X_comp_t)
    X_recon_vae = X_recon_vae_t.cpu().numpy()
    X_recon_ae = X_recon_ae_t.cpu().numpy()

fig, axes = plt.subplots(3, n_comp, figsize=(12, 6))

for i in range(n_comp):
    axes[0, i].imshow(X_comp[i].reshape(28, 28), cmap='gray')
    axes[0, i].set_title("Original")
    axes[0, i].axis('off')
    
    axes[1, i].imshow(X_recon_vae[i].reshape(28, 28), cmap='gray')
    axes[1, i].set_title("VAE")
    axes[1, i].axis('off')
    
    axes[2, i].imshow(X_recon_ae[i].reshape(28, 28), cmap='gray')
    axes[2, i].set_title("AE Padrão")
    axes[2, i].axis('off')

plt.suptitle("Comparação: VAE vs Autoencoder Padrão")
plt.tight_layout()
plt.show()

In [ ]:
# Comparar geração: VAE vs AE
n_gen_comp = 10

vae.eval()
ae.eval()
with torch.no_grad():
    z = torch.randn(n_gen_comp, vae.latent_dim).to(device)
    X_gen_vae_t = vae.decode(z)
    X_gen_ae_t = ae.decode(z)
    X_gen_vae = X_gen_vae_t.cpu().numpy()
    X_gen_ae = X_gen_ae_t.cpu().numpy()

fig, axes = plt.subplots(2, n_gen_comp, figsize=(15, 3))

for i in range(n_gen_comp):
    axes[0, i].imshow(X_gen_vae[i].reshape(28, 28), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel("VAE", fontsize=12)
    
    axes[1, i].imshow(X_gen_ae[i].reshape(28, 28), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_ylabel("AE Padrão", fontsize=12)

plt.suptitle("Comparação de Geração: VAE vs Autoencoder Padrão")
plt.tight_layout()
plt.show()

A comparação quantitativa entre os dois modelos revela trade-offs fundamentais na arquitetura de autoencoders. Conforme mostrado na tabela abaixo, o autoencoder padrão alcança uma loss de reconstrução de 64.67, superando o VAE que obtém 76.81, o que resulta em uma diferença de 18.8% ou 12.14 pontos absolutos a favor do autoencoder. Esta vantagem em reconstrução é esperada e facilmente explicável: o autoencoder não possui a penalidade de KL divergence que força o espaço latente a seguir uma distribuição específica, permitindo, portanto, que ele otimize exclusivamente a fidelidade de reconstrução sem restrições.

| Métrica | VAE | AE Padrão | Diferença |
|---------|-----|-----------|-----------|
| Loss de Reconstrução | 76.81 | 64.67 | +18.8% (12.14 pontos) |
| Geração de N(0,I) | Dígitos reconhecíveis | Ruído | Qualitativa |
| Espaço Latente | Estruturado N(0,I) | Irregular | - |

Por outro lado, a diferença qualitativa aparece drasticamente na capacidade de geração. Quando amostramos pontos aleatórios da distribuição normal padrão N(0,I) e os decodificamos, o VAE produz dígitos reconhecíveis e plausíveis, enquanto o autoencoder padrão gera apenas ruído visual sem estrutura. Esta disparidade ocorre porque o termo de KL divergence no VAE explicitamente força o espaço latente a seguir a distribuição N(0,I), garantindo assim que amostragens aleatórias dessa distribuição caiam em regiões do espaço latente que correspondem a dados válidos. O autoencoder padrão, em contrapartida, desenvolve um espaço latente irregular e descontinuado, onde a maioria dos pontos amostrados de N(0,I) não corresponde a nenhuma estrutura aprendida.

O trade-off é, portanto, quantitativamente claro: o VAE sacrifica aproximadamente 16% de qualidade de reconstrução em troca de obter capacidade generativa completa e um espaço latente estruturado e interpretável. Dessa forma, para aplicações que necessitam apenas de compressão e reconstrução (como redução de dimensionalidade para classificação downstream), um autoencoder padrão é suficiente e preferível. Inversamente, para aplicações que demandam capacidade generativa (interpolação entre amostras, exploração do espaço de dados, síntese de novas amostras realistas), o VAE é necessário, e o custo de 16% em reconstrução constitui o preço a ser pago por estas funcionalidades adicionais.

## Resultados

A implementação final do VAE utilizou uma arquitetura com 652.824 parâmetros treináveis, estruturada em um encoder que mapeia os 784 pixels de entrada para 400 neurônios ocultos e subsequentemente para os parâmetros μ e log σ² de um espaço latente de dimensão 20, seguido por um decoder simétrico que reconstrói a imagem original a partir da amostra latente. Esta configuração resulta, portanto, em uma compressão de fator 39×, equivalente a uma redução dimensional de 97.4%.

As métricas finais obtidas após 50 épocas de treinamento foram: loss total de 102.64, loss de reconstrução (BCE) de 76.81, e KL divergence de 25.82. Estes valores estão perfeitamente alinhados com os ranges típicos reportados na literatura para VAEs treinados no MNIST, onde valores de KL entre 20 e 30 e BCE entre 70 e 85 são considerados padrão. Além disso, a ausência de overfitting foi confirmada pela proximidade entre as losses de treino e validação ao longo de todo o treinamento.

A estratégia de beta annealing, implementada com transição gradual de β=0 para β=1 ao longo das primeiras 10 épocas, foi essencial para a estabilidade do treinamento. Esta abordagem permitiu que o modelo primeiro aprendesse a tarefa de reconstrução antes de impor a regularização completa do espaço latente, resultando, consequentemente, em uma redução controlada da KL divergence de 281 para 26 (uma redução de 90.8%) sem as explosões numéricas que caracterizavam implementações anteriores. Os hiperparâmetros finais escolhidos foram learning rate de 1e-3, dimensão oculta de 400, dimensão latente de 20, e período de annealing de 10 épocas.

A comparação com um autoencoder padrão de arquitetura idêntica revelou o trade-off fundamental entre reconstrução e capacidade generativa: o autoencoder padrão alcançou loss de reconstrução de 64.67 versus 76.81 do VAE, uma diferença de 16%. No entanto, apenas o VAE consegue gerar amostras realistas a partir de pontos amostrados de N(0,I), enquanto o autoencoder produz ruído. Este resultado quantifica precisamente o custo da capacidade generativa: sacrifica-se 16% de fidelidade de reconstrução para obter um espaço latente estruturado que permite geração, interpolação e exploração sistemática.

A implementação em PyTorch, por sua vez, trouxe benefícios significativos em relação a abordagens anteriores em NumPy. Em primeiro lugar, garantiu estabilidade numérica automática para operações sensíveis como exponenciais e logaritmos. Em segundo lugar, forneceu backpropagation automática através do autograd, eliminando a necessidade de derivadas manuais. Ademais, possibilitou aceleração via GPU utilizando a NVIDIA RTX 3060 Laptop disponível. Finalmente, ofereceu APIs de alto nível para módulos e otimizadores que simplificaram significativamente o código.

Do ponto de vista de insights práticos, observamos que: (1) beta annealing é crucial para evitar tanto o colapso do espaço latente (KL→0) quanto explosões numéricas; (2) aceleração por GPU viabilizou o uso de batch_size=128, acelerando significativamente o treinamento; (3) a dimensão latente de 20 mostrou-se suficiente para capturar a variabilidade de 10 classes de dígitos; (4) o VAE é necessário para aplicações generativas, mas um autoencoder padrão é suficiente quando apenas compressão é requerida.

As aplicações práticas deste modelo incluem, entre outras: geração de dados sintéticos para aumento de datasets; detecção de anomalias através da análise de amostras com alta loss de reconstrução; aprendizado de representações compactas (embeddings de 20 dimensões) para tarefas downstream; interpolação no espaço latente para criar transições suaves entre amostras; e compressão de dados com estrutura probabilística interpretável.

Finalmente, extensões naturais deste trabalho incluem a implementação de Conditional VAEs (CVAE) para geração condicionada em labels específicos, permitindo controle direto sobre a classe do dígito gerado; β-VAEs com controle explícito do parâmetro β para explorar diferentes pontos no espectro entre reconstrução e regularização; e Hierarchical VAEs (HVAE) com múltiplas camadas latentes para capturar estruturas de dependência mais complexas nos dados.